# Free Energy Calculations: Methane in Water

This is a tutorial based on [this](http://www.bevanlab.biochem.vt.edu/Pages/Personal/justin/gmx-tutorials/free_energy/index.html) one. We will use BAC2.0

The system contains a single molecule of methane (called "ALAB" in the coordinate file, for the β-carbon of alanine) in a box of 596 TIP3P water molecules.

In [1]:
import bac.simulate.gromacs as gmx
from pathlib import Path

In [2]:
em_steep = gmx.Run(integrator='steep', number_of_steps=5000)
em_steep.name = 'em_steep'
em_steep.coordinates = Path('input_files/methane_water.gro').absolute()
em_steep.topology = Path('input_files/topol.top').absolute()

em_steep.energy_log_output_frequency = 1
em_steep.energy_output_frequency = 100

In [3]:
em_steep.generate_velocities = False

em_steep.minimization_tolerance = 100
em_steep.minimization_step_size = 0.01
em_steep.minimization_correction_steps = 10

In [4]:
nbc = em_steep.non_bonded_controller
nbc.cutoff_scheme = gmx.non_bonded_controller.CutoffSchemeType.verlet
nbc.neighbor_list_update_frequency = 1
nbc.neighbor_list_type = gmx.non_bonded_controller.NeighborListType.grid
nbc.periodic_boundary_condition = gmx.non_bonded_controller.PeriodicBoundaryConditionType.xyz
nbc.cutoff = 1.2

nbc.coulomb_type = gmx.non_bonded_controller.CoulombType.pme
nbc.coulomb_cutoff = 1.2

nbc.van_der_waals_type = gmx.non_bonded_controller.VanDerWaalsType.cutoff
nbc.van_der_waals_modifier = gmx.non_bonded_controller.VanDerWaalsModifierType.potential_shift
nbc.van_der_waals_switch_cutoff = 1.0
nbc.van_der_waals_cutoff = 1.2

nbc.correct_long_range_dispersion = gmx.non_bonded_controller.LongRangeDispersionCorrectionType.energy_and_pressure

nbc.fourier_spacing = 0.12
nbc.pme_order = 6
nbc.ewald_tolerance_coulomb = 1e-06
nbc.ewald_epsilon_surface = 0

In [5]:
fe = gmx.FreeEnergyController()
em_steep.free_energy_controller = fe


# fe.delta_lambda = 0
# fe.calculate_lambda_neighbors = 1

# This will be set by an ensemble_modifier is the workflow has one. There is no need to do it now.
# fe.initial_lambda_state = 0

fe.fep_lambdas = [0]*21
fe.van_der_waals_lambdas = [x*0.05 for x in range(21)] 

fe.soft_core_alpha = 0.5
fe.soft_core_coulomb = False
fe.soft_core_power = 1
fe.soft_core_sigma = 0.3

fe.couple_molecule_groups = 'Methane'
fe.couple_type_initial_lambda = 'vdw'
fe.couple_type_final_lambda = 'none'
fe.couple_intramolecular = False

fe.output_frequency = 10

In [6]:
em_steep.constraints.continuation = False
em_steep.constraints.type = gmx.ConstraintType.h_bonds
em_steep.constraints.algorithm = gmx.ConstraintAlgorithmType.lincs
em_steep.constraints.lincs_order = 12

Next energy minimization step

In [7]:
em_bfgs = next(em_steep)

In [8]:
em_bfgs.integrator = gmx.Integrator.l_bfgs
em_bfgs.constraints.type = gmx.ConstraintType.none
em_bfgs.name = 'em_bfgs'

In [9]:
nvt = next(em_bfgs)
nvt.name = 'nvt'

In [10]:
nvt.integrator = 'sd'
nvt.delta_time = 0.002
nvt.number_of_steps = 50000
nvt.com_motion_removal_frequency = 100

nvt.generate_velocities = True
nvt.generate_temperature = 300

In [11]:
nvt.temperature_controller.groups = 'system'
nvt.temperature_controller.time = 1
nvt.temperature_controller.temperature = 300

## Simulation and Workflow

In [12]:
from bac.simulate import Workflow, replica, lambda_window

In [13]:
wf = Workflow(resource='MPB', name='MeOH_H2O')

In [14]:
wf.add_simulation(em_steep)
wf.add_simulation(em_bfgs)
wf.add_simulation(nvt)

In [15]:
wf.ensembles = [replica(1), lambda_window(1)]

In [16]:
wf.preprocess_simulations(execute=False)

gmx grompp -f replica_0/lambda_0/em_steep.mdp -c /Users/kristofarkas/Developer/BAC2/examples/gromacs_fe/input_files/methane_water.gro -p /Users/kristofarkas/Developer/BAC2/examples/gromacs_fe/input_files/topol.top -o replica_0/lambda_0/em_steep.tpr -po replica_0/lambda_0/em_steep_generated.mdp 
gmx grompp -f replica_0/lambda_0/em_bfgs.mdp -c replica_0/lambda_0/em_steep.gro -p /Users/kristofarkas/Developer/BAC2/examples/gromacs_fe/input_files/topol.top -o replica_0/lambda_0/em_bfgs.tpr -po replica_0/lambda_0/em_bfgs_generated.mdp 
gmx grompp -f replica_0/lambda_0/nvt.mdp -c replica_0/lambda_0/em_bfgs.gro -p /Users/kristofarkas/Developer/BAC2/examples/gromacs_fe/input_files/topol.top -o replica_0/lambda_0/nvt.tpr -po replica_0/lambda_0/nvt_generated.mdp 
